In [1]:
%useLatestDescriptors
%use ggdsl

* [Basics](#Basics)
  * [Data](#Data)
  * [Plot creation](#Plot-creation)
    * [Layers, aesthetics, mappings and scales](#Layers,-aesthetics,-mappings-and-scales)
    * [Scale parameters: axis and legend](#Scale-parameters:-axis-and-legend)
  * [Special aesthetic types](#Special-aes-types)
  * [Groupuing](#Grouping)
  * [Iterable API](#Iterable-API)
* [DataFrame API](#DataFrame-API)
  * [DataFrame x Groupping](#DataFrame-x-Groupping)
* [Multiplot](#Multiplot)
  * [Plot grid](#Plot-grid)
  * [Plot bunch](#Plot-bunch)
  * [Facetting](#Facetting)
* [Position](#Position)
* [Statistics](#Statistics)
* [Series](#Series)
  * [DataFrame x Series](#DataFrame-x-Series)
* [Layout](#Layout)
  * [Title, subtitle, caption and size](#Title,-subtitle,-caption-and-size)
  * [Themes](#Themes)
    * [Custom themes](#Custom-themes)
* [Custom scales](#Custom-scales)
* [Tooltips](#Tooltips)
* [Export](#Export)

# Basics

## Data

The main data model for working with plotting is "named data" or "dataframe", i.e. a set of named value columns of the same size. At the moment, the library does not support working with nullable values. Thus, the input data must have the form `Map<String, List<Any>>`.

In [2]:
val dataset = mapOf(
    "time, ms" to listOf(12, 87, 130, 149, 200, 221, 250),
    "relativeHumidity" to listOf(0.45, 0.3, 0.21, 0.15, 0.22, 0.36, 0.8),
    "flowOn" to listOf(true, true, false, false, true, false, false)
)

In the future we will need to refer to the columns of our dataset. To do this, we need to create a pointer for each one.

In [3]:
// 1. Using `column()` function. We need specify type of column and its name in the dataset:
val timeMs = column<Int>("time, ms")
// 2. String API similar to the previous one, but with using invocation of `String`:
val humidity = "relativeHumidity"<Double>()
// 3. Using delegation of unnamed column - its name will be taken from the name of the variable:
val flowOn by column<Boolean>()

## Plot creation

To create a plot, you need to call the `plot()` function by passing a dataset as an argument. This function creates a context in which you can add layers. A *layer* is a set of mappings from the data to the graph's visual parameters. Here's an example of a graph with one simple layer:

In [4]:
plot(dataset) {
    points {
        // maps values from "time, ms" to X
        x(timeMs)
        // maps values from "relativeHumidity" to Y
        y(humidity)
        // set size of points to 4.5
        size = 4.5
    }
}

### Layers, aesthetics, mappings and scales

Each layer is characterized by its *geometrical entity* or just *geom*. Each geom has its own set of *aesthetic attributes* or simply *aesthetics* or *aes*. Aesthetics can be *positional* (e.g. `x`, `y`, `yMin`, `yMax`, `middle`) or *non-positional* (such as `color`, `size`, `width`). Non-positional attributes are characterized by some type (for example, `size` associated with `Double`, `color` with a special type `Color`).
The aes value can be assigned in 2 ways: by setting and by mapping.

*Setting* is a simply setting constant value:
```
x(12.0f)
size(5.0)
color(Color.RED)
```

*Mapping* is a mapping from the data column to the values of the aesthetic attribute.
```
x(timeMs)
size(humidity)
color(flowOn)
```

The function of this mapping is called *scale*. Scales play a key role in data visualization. In the examples above, the mappings use the default scales, but we can specify the scales explicitly. There are two types of scales  &mdash; *categorical* (or discrete) and *continuous*, depends on its domain and range type. If scale is continuous, its domain and range are set using limits, while categorical scale domain and range are set like lists of categories and corresponding to them values. Also scales can be either positional or non-positional (depending on which aesthetic attributes are displayed). Refined scales (with explicit domain/range) are typed. Continuous scales have `transform` parameter; it defines the function type (which is linear by default). Scales can be created with special functions:

In [5]:
// Non-positional unspecified categorical scale.
val nonPosCatUnspec = categorical()
// Non-positional unspecified continuous scale.
val nonPosContUnspec = continuous(Transformation.LOG10)
// Positional unspecified categorical scale.
val posCatUnspec = categoricalPos()
// Positional unspecified continuous scale.
val posContUnspec = continuousPos()

// Non-positional categorical scale.
val nonPosCat = categorical(listOf(true, false), listOf(Color.RED, Color.BLUE)) //types are inferred
// Non-positional continuous scale.
val nonPosCont = continuous<Double, Double>(rangeLimits = 8.0 to 17.0) // specify range only, need to specify types
// Positional categorical scale.
val posCat = categoricalPos(listOf(1, 2, 4, 8, 16))
// Positional continuous scale.
val posCont = continuousPos(0 to 260, transform = Transformation.REVERSE)

Line_11.jupyter.kts (2:23 - 34) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <DomainType, RangeType> NonPositionalMappingParameters<TypeVariable(DomainType), TypeVariable(RangeType)>.categorical(vararg categoriesToValues: Pair<TypeVariable(DomainType), TypeVariable(RangeType)>): NonPositionalCategoricalScale<TypeVariable(DomainType), TypeVariable(RangeType)> defined in org.jetbrains.kotlinx.ggdsl.dsl
public inline fun <reified DomainType, reified RangeType> NonPositionalMappingParameters<TypeVariable(DomainType), TypeVariable(RangeType)>.categorical(range: List<TypeVariable(RangeType)>? = ..., domain: List<TypeVariable(DomainType)>? = ...): NonPositionalCategoricalScale<TypeVariable(DomainType), TypeVariable(RangeType)> defined in org.jetbrains.kotlinx.ggdsl.dsl
public fun <DomainType> PositionalMappingParameters<TypeVariable(DomainType)>.categorical(categories: List<TypeVariable(DomainType)>? = ...): PositionalCate

To apply scale on column, simply use `.scaled()` extensin function of column and pass your scale as an argument. Note, that for refined scale its `DomainType` must match the type of column.

In [6]:
val flowOnToColor = flowOn.scaled(nonPosCat)

Line_12.jupyter.kts (1:28 - 34) Unresolved reference: scaled
Line_12.jupyter.kts (1:35 - 44) Unresolved reference: nonPosCat

The last thing left is to create a mapping, which is done in the same way as in the case without an explicit scaling  &mdash; aesthetic invocation:

In [7]:
plot(dataset) {
    points {
        x(timeMs) { scale = posContUnspec }
        y(humidity) { scale = continuous(0.0..1.0) }
        size(humidity) { scale = nonPosCont }
        color(flowOnToColor)
        symbol(flowOn) { scale = categorical() }
    }
}

Line_13.jupyter.kts (3:29 - 42) Unresolved reference: posContUnspec
Line_13.jupyter.kts (5:34 - 44) Unresolved reference: nonPosCont
Line_13.jupyter.kts (6:15 - 28) Unresolved reference: flowOnToColor

### Scale parameters: axis and legend

The most important part of reading the charts is the *guides*. They are in fact essentially mini-charts of scales. The guides of positional scales are *axes*, and the guides of non-positional ones are *legends*. Every applied scale has its own default guide. You can customize it through use `with`

In [8]:
// TODO 2 better breaks API
plot(dataset) {
    points {
        x(timeMs) {
            axis.name = "Time from start of counting,\n milliseconds"
        }
        y(humidity) {
            scale = continuous(0.0..1.0)
            axis {
                name = "Relative humidity"
                breaksLabeled(0.0 to "0%", 0.3 to "30%", 0.6 to "60%", 0.9 to "90%")
            }
         }
        size = 12.0
        color(humidity) {
            scale = continuous()
            legend {
                name = ""
                type = colorBar(40.0, 190.0, 15)
                breaks(format = "e")
            }
        }
    }
}

## Special aesthetic types

In addition to the standard types of aesthetic attributes, there are a few with special restrictions. For example, so-called "sub-positional" aesthetic attributes can be mapped, but only with a regular column pointer, not a scaled one (in fact, these aesthetic attributes are a sub-part of another one, and their mappings have the same scale as the "parent" aesthetic). Also, some aesthetics only support the setting, and can not be mapped.

In [9]:
// TODO lets-plot bug (outliers)
plot(
    mapOf(
        "x" to listOf("a", "b", "c"),
        "min" to listOf(0.8, 0.4, 0.6),
        "lower" to listOf(0.9, 1.4, 0.8),
        "middle" to listOf(1.5, 2.4, 1.6),
        "upper" to listOf(1.9, 3.4, 1.7),
        "max" to listOf(3.1, 4.4, 2.6),
    )
) {
    //y(continuousPos(-3.0 to 10.0))
    boxplot {
        x("x"<String>())
        // sub-y aesthetics:
        yMin("min"<Double>())
        lower("lower"<Double>())
        middle("middle"<Double>())
        upper("upper"<Double>())
        yMax("max"<Double>())
        // `fatten` can only be setted:
        fatten = 4.5
    }
}

## Grouping

This part of API allows you to work with grouped data. You can either provide groped data as dataset or perform grouping inside the plot DSL:

In [10]:
val dataset = mapOf(
    "timeG" to listOf(1.0, 2.2, 3.4, 6.6, 2.1, 4.4, 6.0, 1.5, 4.7, 6.7),
    "value" to listOf(112.0, 147.3, 111.1, 200.6, 90.8, 110.2, 130.4, 100.1, 90.0, 121.8),
    "c-type" to listOf("A", "A", "A", "A", "B", "B", "B", "C", "C", "C")
)

val timeG by column<Double>()
val value by column<Double>()
val cType = column<String>("c-type")

In [11]:
plot(dataset.groupBy(cType)) {
    line {
        x(timeG)
        y(value)
    }
}

Line_17.jupyter.kts (1:14 - 21) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: String): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: KProperty<*>): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(vararg cols: Column /* = ColumnReference<*> */, moveToTop: Boolean = ...): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public fun <T> DataFrame<TypeVariable(T)>.groupBy(moveToTop: Boolean = ..., cols: ColumnsSelector<TypeVariable(T), *> /* = ColumnsSelectionDsl<TypeVariable(T)>.(it: ColumnsSelectionDsl<TypeVariable(T)>) -> ColumnSet<*> */): GroupBy<TypeVariable(T), TypeVariable(T)> defined in org.jetbrains.kotlinx.dataframe.api
public f

In [12]:
plot(dataset) {
    groupBy(cType) {
        line {
            x(timeG)
            y(value)
        }
    }
}

When working with grouped data, mappings are allowed only for those columns that are grouping keys

In [13]:
plot(dataset) {
    groupBy(cType) {
        line {
            x(timeG)
            y(value)
            color(cType)
            width = 4.0
        }
    }
}

## Iterable API

Instead of using dataframes and column pointers, you can simply use iterables as a data source. You can map them to aesthetics just like pointers to columns &mdash; via invocation, and apply scales in the same way (using the `.scaled()` extension)

In [14]:
val month = listOf("January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December")
val numberOfDays = listOf(31, 28, 31, 30, 31, 30, 31, 30, 31, 30, 31, 30)
val season = listOf("winter", "winter", "spring", "spring", "spring", "summer", "summer", "summer", "autumn", "autumn", "autumn", "winter")

plot {
    x(month, "month") {
        scale = categorical() 
    }
    bars {
        y(numberOfDays, "number of days")
        fillColor(season, "season") {
            scale = categorical(
                "winter" to Color.BLUE, "spring" to Color.GREEN, "summer" to Color.RED, "autumn" to Color.ORANGE
            )
        }
     }
}

# DataFrame API

Integration with the DataFrame library allows you to use `DataFrame` as a data source and its columns just like `ColumnReference` from the common API (that is can be mapped, scaled, etc.).  With the auto-generation of property columns, you don't need to create them manually

In [15]:
val mpgDF = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
mpgDF.head()

DataFrame: rowsCount = 5, columnsCount = 12

In [16]:
mpgDF.schema()

untitled: Int
manufacturer: String
model: String
displ: Double
year: Int
cyl: Int
trans: String
drv: String
cty: Int
hwy: Int
fl: String
class: String

In [17]:
val mpgInfoPlot = mpgDF.plot { /* DataFrame.() -> Plot */
        points { 
            x(displ) // auto-generated df columns
            y(cty) { scale = continuous(8..34) }
            symbol = Symbol.CIRCLE_FILLED
            color = Color.WHITE
            fillColor(drv)
            size(hwy) { scale = continuous(range = 5.0..15.0) }
        }
}
mpgInfoPlot

Of course, if you don't have an autogenerated property-column, you can create a pointer to it manually:

In [18]:
val mpgCountPlot = mpgDF.groupBy { drv }. aggregate { 
    count() into "count"
}.plot {
  bars {
      x(drv)
      y("count"<Int>())
      alpha = 0.75
  }  
}
mpgCountPlot

## DataFrame x Groupping

Groped dataframe (i.e. `DataFrame` to which `.groupBy()` was applied) also can be used as a dataset, and then inside the DSL you can work with it via the usual API grouping. The only difference is that the `groupKey` field is used to access the grouping keys, and `column` is used for accesing all columns.

In [19]:
// TODO better example
val mpgLinePlot = mpgDF.groupBy { drv }.plot {
  line {
      x(displ)
      y(cty) { scale = continuous(8..34) }
      color(drv)
  }  
}
mpgLinePlot

# Multiplot

There are 3 ways to create a multiplot: `plotGrid`, `plotBunch` and facetting.

## Plot grid

`plotGrid` function arrange plots in cells of a regular grid.

In [20]:
plotGrid(listOf(mpgInfoPlot, mpgCountPlot, mpgLinePlot), 
         nCol = 2, 
         cellWidth = 500,
         cellHeight = 400,
)

Line_28.jupyter.kts (3:10 - 19) Cannot find a parameter with this name: cellWidth
Line_28.jupyter.kts (4:10 - 20) Cannot find a parameter with this name: cellHeight

## Plot bunch

Collection of plots without an automatic layout. Each plot can have arbitrary location and size.

In [21]:
plotBunch { 
    add(mpgCountPlot, 0, 0, 400, 200)
    add(mpgLinePlot, 400, 0, 300, 200)
    add(mpgInfoPlot, 0, 200, 700, 300)
}

## Facetting

Facetting creates a multiplot from one plot by splitting dataset (like `groupBy`). Facetting can be applied by invocation `facetGridX`, `facetGridY` (for creating facetting by 1 parameter along `X` and `Y` axes correspondingly), `facetGrid`(for creating facetting by 2 parameter along `X` and `Y` axes) and `facetWrap` (for creating facetting by multiple parameteres).

In [22]:
mpgDF.plot { 
    points { 
        x(displ)
        y(cty) { scale = continuous(8..34) }

        symbol = Symbol.CIRCLE_FILLED
        color = Color.WHITE
        fillColor(drv)
        size(hwy) { scale = continuous(range = 5.0..15.0) }
    }

    layout.size = 750 to 450

    facetGridX(drv, scalesSharing = ScalesSharing.FREE_X)
}

In [23]:
mpgDF.plot { 
    points { 
        x(displ)
        y(cty) { scale = continuous(8..34) }

        symbol = Symbol.CIRCLE_FILLED
        color = Color.WHITE
        fillColor(drv)
        size(hwy) { scale = continuous(range = 5.0..15.0) }
    }

    layout.size = 750 to 450

    facetGridY(cyl)
}

In [24]:
mpgDF.plot { 
    points { 
        x(displ)
        y(cty) { scale = continuous(8..34) }

        symbol = Symbol.CIRCLE_FILLED
        color = Color.WHITE
        fillColor(drv)
        size(hwy) { scale = continuous(range = 5.0..15.0) }
    }

    layout.size = 750 to 450

    facetGrid(drv, cyl)
}

In [25]:
//TODO letsplot bug?
mpgDF.plot { 
    points { 
        x(displ)
        y(cty) { scale = continuous(8..34) }

        symbol = Symbol.CIRCLE_FILLED
        color = Color.WHITE
        fillColor(drv)
        size(hwy) { scale = continuous(range = 5.0..15.0) }
    }

    layout.size = 750 to 450

    facetWrap(nCol = 3, scalesSharing = ScalesSharing.FREE){
        facet(drv)
        facet(cyl, order = OrderDirection.DESCENDING)
    }
}

# Position

`position` parameter adjusts the relative position of objects from different groups within the same layer.

In [26]:
mpgDF.groupBy {cyl}.plot {
   bars {
       x(drv)
       y(cty)
       fillColor(cyl)
       
       // default
       position = Position.Identity
   }
}

In [27]:
mpgDF.groupBy {cyl}.plot {
   bars {
       x(drv)
       y(cty)
       fillColor(cyl) { scale = categorical() }
       
       position = Position.Dodge(0.9)
   } 
}

In [28]:
mpgDF.groupBy {cyl}.plot {
   bars {
       x(drv)
       y(cty)
       fillColor(cyl) { scale = categorical() }
       
       position = Position.Stack
   } 
}

*Note*: `position` works only inside one layer, so it is impossible to configure the relative position of objects from different layers. However, this restriction may be particuarry circumvented by using series plots, see **Series**.

# Statistics

Rather than using statistical pre-transformations of your dataset, you can calculate it inside the DSL. The "stat" family of functions is used for this purpose. These functions convert the original data into a new dataset with calculated statistics. The set of these statistics is defined by the function. Within a context, its statistics can be accessed through the `Stat` field.These statistics act as pointers to the columns in the new dataset. And you can do the same thing with them as with regular `ColumnReference` : they can be mapped to aesthetics, scaled, used in tooltips, etc.

In [29]:
import kotlin.random.Random

val observations = List(1000) { Random.nextDouble() }
val observationsDataset = mapOf(
    "observations" to observations
)
val obs = column<Double>("observations")

In [30]:
plot(observationsDataset) {
    statBin(obs) {
         bars {
            // simple mapping
            x(Stat.BINS)
            // mapping with scale
            y(Stat.COUNT) { scale = continuous(0.0..100.0, transform = Transformation.REVERSE) }

            alpha = 0.5

            // formatting of stat value format
            tooltips(statFormats = mapOf(
                Stat.COUNT to "d"
            )) {
                // line with the name of stat (i.e. "count") on the left side and its value on the right side
                line(Stat.COUNT)
             }
         }

         path {
             x(Stat.BINS)
             y(Stat.COUNT)

             width = 2.5
             color = Color.RED
         }
    }
}

Line_38.jupyter.kts (12:22 - 33) Cannot find a parameter with this name: statFormats

In addition, for basic statistical charts there is a simpler API, which combines into one function the counting of statistics and the creation of a layer. For example, a histogram is nothing more than a counting of "bin" statistics and a bar chart that has bins values on X and count values on Y.

In [31]:
val histPlot = plot(observationsDataset) {
    histogram(obs)

    layout.title = "`histogram`"
}
histPlot

You can compare it to a bar chart with the calculation of bins stat:

In [32]:
val binBarPlot = plot(observationsDataset) {
    statBin(obs) {
        bars {
            x(Stat.BINS)
            y(Stat.COUNT)
         }
    }
    layout.title = "`statBin` + `bar`"
}

plotGrid(listOf(histPlot, binBarPlot), 2, 800, 600)

Line_40.jupyter.kts (11:43 - 46) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot
Line_40.jupyter.kts (11:48 - 51) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot

`histogram` function also opens a context, in which you can create bindings for the aesthetic attributes of the bar. In addition, the "stat-bin" stats are defined in this context, allowing you to map them to aesthetics. Also, you can rewrite default mappings to coordinates (to display relative values instead of absolute ones, for example).

In [33]:
plot(observationsDataset) {
    histogram(obs, Bins.byWidth(0.05), BinXPos.center(0.5)) {
        y(Stat.DENSITY)

        color(Stat.COUNT) { scale = continuous(range = Color.GREEN to Color.RED) }

        borderLine {
            color = Color.BLACK
            width = 0.3
        }

        tooltips(title = "${value(Stat.BINS)} ± 0.025") {
            line(Stat.DENSITY)
        }
    }
}

Line_41.jupyter.kts (5:9 - 14) Unresolved reference: color
Line_41.jupyter.kts (5:29 - 34) Unresolved reference: scale
Line_41.jupyter.kts (5:37 - 47) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun <DomainType, RangeType> NonPositionalMappingParameters<TypeVariable(DomainType), TypeVariable(RangeType)>.continuous(rangeMin: TypeVariable(RangeType)? = ..., rangeMax: TypeVariable(RangeType)? = ..., domainMin: TypeVariable(DomainType)? = ..., domainMax: TypeVariable(DomainType)? = ..., nullValue: TypeVariable(RangeType)? = ..., transform: NonPositionalTransform? = ...): NonPositionalContinuousScale<TypeVariable(DomainType), TypeVariable(RangeType)> defined in org.jetbrains.kotlinx.ggdsl.dsl
public fun <DomainType, RangeType : Comparable<TypeVariable(RangeType)>> NonPositionalMappingParameters<TypeVariable(DomainType), TypeVariable(RangeType)>.continuous(range: ClosedRange<TypeVariable(RangeType)>, nullValue: TypeVariable(

Stats API works with `Iterable` as well:

In [34]:
plotGrid(listOf(
    plot {
        statBin(observations) {
            points {
                x(Stat.DENSITY)
                y(Stat.BINS)
            }
        }
    },
    plot {
        histogram(observations)
    }
), 2, 800, 600)

Line_42.jupyter.kts (13:7 - 10) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot
Line_42.jupyter.kts (13:12 - 15) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot

# Series

In addition to the classic GoG-like approach, you can also use a more familiar approach, the so-called *series* approach (which is used in matplotlib, plotly and many other plotting libraries). That is, you can create plot by defining not layers, but series. Series are similar to layers, but unlike layers, data in a series can only be mapped on coordinates (non-position attributes can still be set as well). Every series has its own label. Under the hood, mappings are created from the set of labels of the series of this plot to those non-positional attributes that have been set within the series (note the limitation: settings for the same attributes must be made within the series!; TODO: autocomplete?)

In [35]:
import kotlin.random.Random

val time = List(15) {it}
val valueA = List(15) { Random.nextDouble() }
val valueB = List(15) { Random.nextDouble() }

//TODO
val dataSeries = mapOf(
    "time" to time,
    "valueA" to valueA,
    "valueB" to valueB
)

val timeSrc = column<Int>("time")
val valASrc = column<Int>("valueA")
val valBSrc = column<Int>("valueB")

Series-plot can be created by using corresponding functions. For example, `barPlot`:

In [36]:
linePlot(dataSeries) {
    x(timeSrc)
    // general setting for all series
    width(2.5)
    series("A1") {
        y(valASrc)
        color(Color.RED)
        type(LineType.DASHED)
    }
    series("B2+") {
        y(valBSrc)
        color(Color.ORANGE)
        type(LineType.SOLID)
    }
}

Line_44.jupyter.kts (1:1 - 9) Unresolved reference: linePlot
Line_44.jupyter.kts (2:5 - 6) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun PlotContext.x(column: String, parameters: LetsPlotPositionalMappingParameters<Any?>.() -> Unit = ...): PositionalMapping<Any?> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(values: Iterable<TypeVariable(T)>, name: String? = ..., parameters: LetsPlotPositionalMappingParameters<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(values: DataColumn<TypeVariable(T)>, parameters: LetsPlotPositionalMappingParameters<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(column: ColumnReference<TypeVariable(T)>, parameters: LetsPlotPositionalMappingParameters<TypeVariable(T

Or simply use `Iterable`:

In [37]:
barPlot(position = Position.Dodge(0.8)) {
    x(time)
    // general setting for all series
    width(0.5)
    series("A1") {
        y(valueA)
        color(Color.RED)
    }
    series("B2+") {
        y(valueB)
        color(Color.ORANGE)
    }
}

Line_45.jupyter.kts (1:1 - 8) Unresolved reference: barPlot
Line_45.jupyter.kts (2:5 - 6) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun PlotContext.x(column: String, parameters: LetsPlotPositionalMappingParameters<Any?>.() -> Unit = ...): PositionalMapping<Any?> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(values: Iterable<TypeVariable(T)>, name: String? = ..., parameters: LetsPlotPositionalMappingParameters<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(values: DataColumn<TypeVariable(T)>, parameters: LetsPlotPositionalMappingParameters<TypeVariable(T)>.() -> Unit = ...): PositionalMapping<TypeVariable(T)> defined in org.jetbrains.kotlinx.ggdsl.letsplot
public fun <T> PlotContext.x(column: ColumnReference<TypeVariable(T)>, parameters: LetsPlotPositionalMappingParameters<TypeVariable(T)

## DataFrame x Series

Series plot can be created with DataFrame API:

In [38]:
val seriesDF = dataFrameOf(
    "time" to time,
    "aVal" to valueA,
    "bVal" to valueB,
)

In [39]:
seriesDF.plot { pointPlot {
   size(6.0)
   series("A") {
       x(time)
       y(aVal)

       symbol(Symbol.ASTERIX)
       color(Color.hex(0xffcc99))
   }
   series("B") {
       x(time)
       y(bVal)

       symbol(Symbol.DIAMOND)
       color(Color.BLUE)
   }
} }

Line_49.jupyter.kts (1:17 - 26) Unresolved reference: pointPlot
Line_49.jupyter.kts (2:4 - 8) Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public fun AnyFrame /* = DataFrame<*> */.size(): DataFrameSize defined in org.jetbrains.kotlinx.dataframe
Line_49.jupyter.kts (3:4 - 10) Unresolved reference: series
Line_49.jupyter.kts (7:8 - 14) Unresolved reference: symbol
Line_49.jupyter.kts (8:8 - 13) Unresolved reference: color
Line_49.jupyter.kts (10:4 - 10) Unresolved reference: series
Line_49.jupyter.kts (14:8 - 14) Unresolved reference: symbol
Line_49.jupyter.kts (15:8 - 13) Unresolved reference: color

# Layout

## Title, subtitle, caption and size

They plot `title`, `subtitle`, `caption` and `size` be configured via `layout` (of which properties they are). Layout fields can be accessed directly or via invocation:

In [40]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
    }
    
    // can acces of layout parameters in this context
    layout {
        subtitle = "plot subtitle"
        caption = "plot caption \n important info"
        size = 800 to 600
    }
    // if you just want to put a title
    layout.title = "PLOT TITLE"
}

## Themes

Themes allow you to customize all the graphical elements of the layout &mdash; styles of lines, backgrounds, text, etc. You can create your own theme or use one of the prepared.

To apply theme, just use `theme()` method of `layout`:

In [41]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
    }
    layout.theme(Theme.Classic)
}

In [42]:
import org.jetbrains.kotlinx.ggdsl.ir.Plot
fun plotWithTheme(theme: Theme? = null, title: String? = null):Plot {
    return mpgDF.plot {
        points {
            x(cty)
            y(hwy)
        }
        layout {
            theme?.let {
                theme(it)
            }
            this.title = title
        }
    }
}

In [43]:
plotGrid(listOf(
    plotWithTheme(Theme.Classic, "\"classic\" theme"),
    plotWithTheme(Theme.Grey, "\"grey\" theme"),
    plotWithTheme(Theme.Light, "\"light\" theme"),
    plotWithTheme(Theme.Minimal, "\"minimal\" theme"),
    plotWithTheme(Theme.Minimal2, "\"minimal2\" theme (by default)"),
    plotWithTheme(Theme.None, "\"none\" theme"),
), 2, 600, 400)

Line_53.jupyter.kts (8:7 - 10) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot
Line_53.jupyter.kts (8:12 - 15) Too many arguments for public fun plotGrid(items: List<Plot>, nCol: Int? = ...): PlotGrid defined in org.jetbrains.kotlinx.ggdsl.letsplot.multiplot

### Custom themes

There is a DSL for creating custom themes. The main part of this DSL is the setting of parameters of type "line", "text" and "background". These parameters can be created separately and then applied, or be set up inplace. Each of them has a `blank` parameter of `Boolean` type. If you set it to true, the item will not be displayed.

In [44]:
val redLine = LayoutParameters.line(Color.RED)

val simpleCustomTheme = theme {
    // use previously created parameters
    xAxis.line(redLine)
    // set up parameters
    yAxis.line {
        color = Color.RED
        width = 0.3
    }
    // remvoe ticks on both axes
    axis.ticks {
        blank = true
    }
}

plotWithTheme(simpleCustomTheme)

In [45]:
// blanking all details on axes:
val blankAxesTheme = theme {
    blankAxes()
}
plotWithTheme(blankAxesTheme)

In [46]:
// TODO theme example notebook reference link????

# Custom scales

In addition to the usual scales, the Let's-Plot provides several custom color scales: `hue`, `grey`, `brewer`, `gradient2` and `gradientN`.

In [47]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        size = 5.0
        color(drv) { scale = categoricalColorHue() }
    }
}

In [48]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        size = 5.0
        color(cty) { scale = continuousColorGrey() }
    }
}

In [49]:
// lp bug?
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        size = 5.0
        color(hwy) {
            scale = continuousColorGradient2(
                Color.YELLOW, Color.BLUE, Color.RED, 1.0,
                domainLimits = 10 to 40
            )
        }
    }
}

In [50]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        size = 5.0
        color(cty) { 
            scale = continuousColorGradientN(
                rangeColors = listOf(Color.RED, Color.hex("#fa89c7"), Color.rgb(200, 89, 12), Color.LIGHT_GREEN)
            )
        }
    }
}

In [51]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        size = 5.0
        color(drv) { scale = categoricalColorBrewer(BrewerType.Qualitative()) }
    }
}

# Tooltips

In addition to aesthetics, data in the visual object can be displayed using tooltips. They are set up in the layer context via a `tooltips()` method. By default tooltips are enabled but you can turn them off setting the `hide` flag to `true`:

In [52]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        color(drv)
        size = 3.5
        
        tooltips(hide = true) {}
    }
}

Pass a list of pointers to the column to display their value for this object. You can customize the format of the value output for the column

In [53]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        color(drv)
        size = 3.5
        
        tooltips(listOf(drv, cyl, displ), formats = mapOf(displ to ".1g")) {}
    }
}

You can customize the title of the tooltips window, as well as its minimum width and fix its position

In [54]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        color(drv)
        size = 3.5
        
        tooltips(
            listOf(cyl, displ),
            title = "Car info",
            anchor = Anchor.TOP_RIGHT,
            minWidth = 80.0,
        ) {}
    }
}

You can also use a special DSL to manually adjust the tooltips lines. The `line` and `value` families of functions are used for this purpose. `line` methods add lines to the tooltips within the context opened by the `tooltips` function, while `value` methods allow you to add a value from a column (or corresponding aesthetic) to the `String`.

In [55]:
mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        color(drv)
        size = 3.5
        
        tooltips(
            title = "${value(manufacturer)} ${value(model)}", // values of columns can be used in the title
            minWidth = 100.0,
            formats = mapOf(cty to "d", hwy to "d")
        ) {
            // standard line (name of the column on the left side and corresponding value on the right side)
            line(displ)
            // similiar to previous one, but we use an aes instead of a column
            line(color)
            // solid line
            line("${value(trans)}")
            // two-sided line
            line("cty/hwy [mpg]", "${value(cty)}/${value(hwy)}") 
        }
    }
}

Line_65.jupyter.kts (16:13 - 17) None of the following functions can be called with the arguments supplied: 
public final fun line(string: String): Unit defined in org.jetbrains.kotlinx.ggdsl.letsplot.tooltips.context.LayerTooltipsContext
public final fun line(leftSide: String? = ..., rightSide: String? = ...): Unit defined in org.jetbrains.kotlinx.ggdsl.letsplot.tooltips.context.LayerTooltipsContext
public final fun line(column: ColumnReference<*>): Unit defined in org.jetbrains.kotlinx.ggdsl.letsplot.tooltips.context.LayerTooltipsContext

# Export

`Plot` can be saved with the `.save()` extension method

In [56]:
val plotForExport = mpgDF.plot {
    points {
        x(cty)
        y(hwy)
        color(drv)
        size = 3.5
    }
    layout {
        title = "Plot for export"
        size = 600 to 400
    }
}
plotForExport

In [57]:
plotForExport.save("plot.png", scale = 2.5, dpi = 9000, path = "./saved_plots")

/Users/pavel.gorgulov/Projects/main_project/ggdsl/examples/notebooks/lets-plot/saved_plots/plot.png

In [58]:
HTML("""<img src="./saved_plots/plot.png" width="600" height="400">""")